In [1]:
import djitellopy

In [3]:
import cv2
import dlib
from djitellopy import Tello
import numpy as np
import time
import face_recognition
width, height = 360, 240
Forward_Backward_Range = [3000, 5000]
pid = [0.4,0.4,0]
pError = 0
drone = Tello()
drone.connect()

print(drone.get_battery())
battery = (drone.get_battery())  # We want to learn battery from drone
if battery > 50:
    drone.streamon()  # Turn on video streaming
    drone.takeoff()
    drone.move_up(30)
else:
    print("please charge it")


[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[WARNING] tello.py - 447 - Aborting command 'command'. Did not receive a response after 7 seconds
[INFO] tello.py - 437 - Send command: 'command'
[WARNING] tello.py - 447 - Aborting command 'command'. Did not receive a response after 7 seconds
[INFO] tello.py - 437 - Send command: 'command'
[WARNING] tello.py - 447 - Aborting command 'command'. Did not receive a response after 7 seconds


Exception: Command 'command' was unsuccessful for 4 tries. Latest response:	'Aborting command 'command'. Did not receive a response after 7 seconds'

In [4]:
def Face_Detection(img):
    obama_image = face_recognition.load_image_file("obama.jpg")
    obama_face_encoding = face_recognition.face_encodings(obama_image)[0]

# Load a second sample picture and learn how to recognize it.
    biden_image = face_recognition.load_image_file("biden.jpg")
    biden_face_encoding = face_recognition.face_encodings(biden_image)[0]

# Create arrays of known face encodings and their names
    known_face_encodings = [
        obama_face_encoding,
        biden_face_encoding
    ]
    known_face_names = [
        "Barack Obama",
        "Joe Biden"
    ]

    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face enqcodings in the frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

        name = "Unknown"

        # If a match was found in known_face_encodings, just use the first one.
        # if True in matches:
        #     first_match_index = matches.index(True)
        #     name = known_face_names[first_match_index]

        # Or instead, use the known face with the smallest distance to the new face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
        
         cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

In [6]:
def trackFace(faceInfo,w,pid,pError):
    area = faceInfo[1]
    x, y = faceInfo[0]
    fb = 0
    error_x = x - w // 2  # w= width; finding center of face
    speed_x = pid[0] * error_x + pid[1] * (error_x - pError)   # determining the yaw
    speed_x = int(np.clip(speed_x, -100, 100))
​
​
    if area > Forward_Backward_Range[0] and area < Forward_Backward_Range[1]:
        fb = 0  #Speed_x değerinde yaw haraketi yaparak yüzü arıyor.
​
    elif area > Forward_Backward_Range[1]:
        fb = -25
        #Yüz yakınlaştıkça geri gidiyor
​
    elif area < Forward_Backward_Range[0] and area != 0:  # If the drone is too far get closer
        fb = 25
        # Yüz uzaklaştıkça ileri gidiyor
​
    elif area ==0:
       drone.rotate_clockwise(20)
       time.sleep(0.8)
​
​
​
    if x == 0:
        speed_x = 0
        error_x = 0
    drone.send_rc_control(0, fb, 0, speed_x)
    print(speed_x, fb)
    return error_x


NameError: name 'frame' is not defined

In [ ]:
while True:
    frame = drone.get_frame_read().frame
    img = cv2.resize(frame, (width, height))


    img,info = Face_Detection(img)
    pError = trackFace(info,width,pid,pError)

    cv2.imshow("Drone Video:", img)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        drone.land()
        break


cv2.destroyAllWindows()